In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

# Gravitational parameter (mu) and mean motion (n)
mu = 398600.4418  # km^3/s^2 (gravitational parameter of Earth)
a = 7000  # km (semi-major axis of the target's orbit)
n = np.sqrt(mu / a**3)  # mean motion of the target

# Initial conditions in the target's orbit reference frame
r0_chaser = np.array([0.0, 0.0, 0.0])  # Initial relative position of chaser, m
v0_chaser = np.array([0.1, 0.1, 0.1])    # Initial relative velocity of chaser, m/s

# Time parameters
t_final = 500  # Total simulation time, seconds (adjusted for orbital periods)
dt = .1   # Time step, seconds

# Time array
t = np.arange(0, t_final, dt)

# Position and velocity arrays
r_chaser = np.zeros((len(t), 3))
v_chaser = np.zeros((len(t), 3))

# Set initial conditions
r_chaser[0] = r0_chaser
v_chaser[0] = v0_chaser

# Clohessy-Wiltshire equations for relative motion
def clohessy_wiltshire(r, v, t, n):
    x, y, z = r
    x_dot, y_dot, z_dot = v
    
    x_ddot = 3 * n**2 * x + 2 * n * y_dot
    y_ddot = -2 * n * x_dot
    z_ddot = -n**2 * z
    
    return np.array([x_dot, y_dot, z_dot]), np.array([x_ddot, y_ddot, z_ddot])

# Simulation loop
for i in range(1, len(t)):
    # Get current state
    r = r_chaser[i-1]
    v = v_chaser[i-1]
    
    # Find accelerations using Clohessy-Wiltshire equations
    v, a = clohessy_wiltshire(r, v, t[i], n)
    
    # Update position and velocity
    r_chaser[i] = r + v * dt
    v_chaser[i] = v + a * dt


#Interative Visualization
# Plotting with Plotly for interactive visualization
import plotly.graph_objects as go
# Create a trace for the chaser
trace = go.Scatter3d(
    x=r_chaser[:, 0],
    y=r_chaser[:, 1],
    z=r_chaser[:, 2],
    mode='markers',
    name='Chaser Satellite'
)
# Create a trace for the target
target_trace = go.Scatter3d(
    x=[0],
    y=[0],
    z=[0],
    mode='markers',
    name='Origin',
    marker=dict(
        color='red',
        size=5
    )
)
# Create the layout
layout = go.Layout(
    title='Relative Motion of the Chaser Satellite',
    scene=dict(
        xaxis=dict(title='X [m]'),
        yaxis=dict(title='Y [m]'),
        zaxis=dict(title='Z [m]')
    )
)
# Create the figure and add the traces
fig = go.Figure(data=[trace, target_trace], layout=layout)
# Show the figure
fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Gravitational parameter (mu) and mean motion (n)
mu = 398600.4418  # km^3/s^2 (gravitational parameter of Earth)
sma = 7000  # km (semi-major axis of the target's orbit)
n = np.sqrt(mu / sma**3)  # mean motion of the target

# Initial conditions in the target's orbit reference frame
r0_chaser = np.array([100000.0, 9000000.0, -3000000.0])  # Initial relative position of chaser, km
v0_chaser = np.array([30000, 500, 0.1])  # Initial relative velocity of chaser, km/s
Initial_mass = 5000  # Initial mass of the chaser, kg

# Thrust parameters
thrust_start_time = 2000  # Time delay before thrust starts in seconds
thrust_force = np.array([0, -6000000, 0])  # Thrust force in N
thrust_acceleration = thrust_force / Initial_mass  # Thrust acceleration in m/s^2
thrust_acceleration = thrust_acceleration * 1e-3 # Convert to km/s^2
print(thrust_acceleration)

# Time parameters
t_final = 50000  # Total simulation time, seconds
dt = 1  # Time step, seconds

# Time array
t = np.arange(0, t_final, dt)

# Position and velocity arrays
r_chaser = np.zeros((len(t), 3))
v_chaser = np.zeros((len(t), 3))

# Set initial conditions
r_chaser[0] = r0_chaser
v_chaser[0] = v0_chaser

# Clohessy-Wiltshire equations for relative motion
def clohessy_wiltshire(r, v, t, n):
    x, y, z = r
    x_dot, y_dot, z_dot = v
    
    x_ddot = 3 * n**2 * x + 2 * n * y_dot
    y_ddot = -2 * n * x_dot
    z_ddot = -n**2 * z
    
    return np.array([x_dot, y_dot, z_dot]), np.array([x_ddot, y_ddot, z_ddot])

# Simulation loop
for i in range(1, len(t)):
    # Get current state
    r = r_chaser[i-1]
    v = v_chaser[i-1]
    
    # Find accelerations using Clohessy-Wiltshire equations
    v, a = clohessy_wiltshire(r, v, t[i], n)
    
    # Apply thrust if within thrust duration and fuel is available
    if thrust_start_time <= t[i]:
    # Total acceleration
        v = thrust_acceleration

    # Update position and velocity
    r_chaser[i] = r + v * dt
    v_chaser[i] = v + a * dt

# Plotting with Plotly for interactive visualization
r_chaser_scaled=np.divide(r_chaser,sma)

# Create a trace for the chaser
trace = go.Scatter3d(
    x=r_chaser_scaled[:, 0],
    y=r_chaser_scaled[:, 1],
    z=r_chaser_scaled[:, 2],
    mode='markers',
    name='Chaser Satellite',
    marker=dict(
        color='blue',
        size=2
    )
)

#create marker for inital position
initial_trace = go.Scatter3d(
    x=[r_chaser_scaled[0, 0]],
    y=[r_chaser_scaled[0, 1]],
    z=[r_chaser_scaled[0, 2]],
    mode='markers',
    name='Initial Position',
    marker=dict(
        color='black',
        size=5
    )
)

# Create a trace for the target
target_trace = go.Scatter3d(
    x=[0],
    y=[0],
    z=[0],
    mode='markers',
    name='Target Satellite',
    marker=dict(
        color='red',
        size=5
    )
)

# Create marker for start of thrust
thrust_start_trace = go.Scatter3d(
    x=[r_chaser_scaled[thrust_start_time, 0]],
    y=[r_chaser_scaled[thrust_start_time, 1]],
    z=[r_chaser_scaled[thrust_start_time, 2]],
    mode='markers',
    name='Thrust Start',
    marker=dict(
        color='green',
        size=5
    )
)



# Create the layout
layout = go.Layout(
    title='Relative Motion of the Chaser Satellite',
    scene=dict(
        xaxis=dict(title='X/a [km]'),
        yaxis=dict(title='Y/a [km]'),
        zaxis=dict(title='Z/a [km]')
    )
)

# Create the figure and add the traces
fig = go.Figure(data=[trace, target_trace, thrust_start_trace, initial_trace], layout=layout)

# Show the figure
fig.show()